<a href="https://www.nvidia.com/dli"> <img src="images/DLI_Header.png" alt="Header" style="width: 400px;"/> </a>

# 图神经网络入门

## 01 - 图和图神经网络简介 ##
在本 Notebook 中，您将学习图和图神经网络的基本概念。

**目录**
<br>
本 Notebook 包括以下部分：
1. [图简介](#s1-1)
    * [处理图数据](#s1-1.1)
    * [练习 #1 - 构建一个简单的图](#s1-e1)
    * [处理节点特征和边特征](#s1-1.2)
    * [图数据的表示](#s1-1.3)
2. [数据集概述](#s1-2)
    * [探索性地分析数据](#s1-2.1)
    * [练习 #2 - 查找连接数最多的节点](#s1-e2)
    * [数据准备和子图](#s1-2.2)
3. [构建用于节点分类的图神经网络](#s1-3)
    * [消息传递和图卷积](#s1-3.1)
4. [使用 PyTorch 构建 GNN](#s1-4)
    * [求和池化](#s1-4.1)
    * [基准 MLP 模型](#s1-4.2)
    * [练习 #3 - 平均池化](#s1-e3)
    * [图卷积网络 (GCN)](#s1-4.3)
5. [使用 DGL 的内置模组构建 GNN](#s1-5)
    * [GraphConv](#s1-5.1)

<a name='s1-1'></a>
## 图简介 ##
图是一种包含**节点**和**边**的数据结构。节点可以是人员、地点或事物；边用于定义节点之间的关系。边可以是有向的，其中每个边都有一个源节点和一个目标节点。边也可以是无向的，没有源节点或目标节点。图在处理复杂的关系和交互问题方面表现出色。

能够自然地以图表示的数据包括：
* **引文网络**可用于研究出版物之间的关系。
* **社交网络**这一工具可用于研究人员、机构和组织的集体行为模式。社交网络图可表示人员群组，方法是将人员建模为节点，并将他们的社交关系建模为边。
* **分子**可描述为图，其中节点是原子，边是共价键。

图的结构可能会因节点的数量、边的数量和节点连接情况不同而大相径庭。

使图有别于其他类型数据的部分属性包括：
1. 图存在于非欧几里德空间中，这增加了解读数据的难度。为直观呈现数据，市面上提供了各种降维工具以供选择。
2. 图是非结构化的，并没有固定的形式。
3. 图的大尺寸和高维度特征增加了人类解读此类数据的复杂性。

<a name='s1-1.1'></a>
### 处理图数据 ###
在本实验中，我们将使用开源 Deep Graph Library [(DGL)](https://www.dgl.ai/) 和 [PyTorch](https://pytorch.org/) 处理图数据。其他热门的图深度学习库包括 [Spektral](https://graphneural.network/)、[Graph Nets](https://www.deepmind.com/open-source/graph-nets) 和 [PyTorch Geometric](https://www.pyg.org/)，这些库让我们能够以类似方式处理图数据。

DGL 使用具有唯一性的整数（也称为节点 ID）来表示每个节点，并使用与端点节点的 ID 对应的整数对来表示每条边。DGL 会根据边添加至图的顺序，给每条边分配具有唯一性的整数（从 `0` 开始，也称为边 ID）。在 DGL 中，所有边都是有向的，边 `(u, v)` 表示它是从源节点 `u` 指向目标节点 `v` 的边。创建**无向**图时，可以通过添加**反向边**，将边视为**双向**边。

首先，我们创建下方的示例图：

<p><img src='images/sample_graph_1.png' width=240></p>

您可以使用 [`dgl.graph(data)`](https://docs.dgl.ai/en/0.9.x/generated/dgl.graph.html) 创建 [`DGLGraph`](https://docs.dgl.ai/en/latest/api/python/dgl.DGLGraph.html)，并使用一对节点 ID `(U, V)` 来表示源节点和目标节点。完成创建后，我们可以使用 [`dgl.DGLGraph.nodes()`](https://docs.dgl.ai/en/0.8.x/generated/dgl.DGLGraph.nodes.html)、[`dgl.DGLGraph.edges()`](https://docs.dgl.ai/en/0.8.x/generated/dgl.DGLGraph.edges.html) 或 [`dgl.DGLGraph.edge_ids(u, v)`](https://docs.dgl.ai/generated/dgl.DGLGraph.edge_ids.html) 引用节点和边。我们还可以使用 [`dgl.DGLGraph.find_edges(eid)`](https://docs.dgl.ai/en/0.8.x/generated/dgl.DGLGraph.find_edges.html)、[`dgl.DGLGraph.in_edges(v)`](https://docs.dgl.ai/en/0.8.x/generated/dgl.DGLGraph.in_edges.html) 或 [`dgl.DGLGraph.out_edges(u)`](https://docs.dgl.ai/en/0.8.x/generated/dgl.DGLGraph.out_edges.html) 识别边。要确定节点度（即连接数），我们可以使用 [`dgl.DGLGraph.in_degrees(v)`](https://docs.dgl.ai/en/0.8.x/generated/dgl.DGLGraph.in_degrees.html) 或 [`dgl.DGLGraph.out_degrees(u)`](https://docs.dgl.ai/en/0.8.x/generated/dgl.DGLGraph.out_degrees.html)。我们可以选择使用 [`networkx`](https://networkx.org/) 直观呈现小图。

_注意：如需进一步了解 DGLGraph API（包括[查询基本图结构](https://docs.dgl.ai/en/0.7.x/api/python/dgl.DGLGraph.html#querying-graph-structure)属性），请单击 [此处](https://docs.dgl.ai/en/0.7.x/api/python/dgl.DGLGraph.html)。_

In [ ]:
# DO NOT CHANGE THIS CELL
# import dependencies
import dgl
import torch

In [ ]:
# DO NOT CHANGE THIS CELL
# create source nodes for edges (2, 1), (3, 2), (4, 3)
sample_u=[2, 3, 4]

# create destination nodes for edges (2, 1), (3, 2), (4, 3)
sample_v=[1, 2, 3]

# create graph
sample_g=dgl.graph((sample_u, sample_v))

# print graph
print(sample_g)

In [ ]:
# DO NOT CHANGE THIS CELL
# print node IDs
print("Node IDs are: \n{}\n".format(sample_g.nodes()))

# print the source and destination nodes of every edge
print("Source & destination nodes of every edge are: \n{}\n".format(sample_g.edges()))

# print edge IDs
print("Edge IDs are: \n{}".format(sample_g.edge_ids(sample_u, sample_v)))

In [ ]:
# DO NOT CHANGE THIS CELL
# make bidirected graph
sample_g=dgl.to_bidirected(sample_g)

# dgl.add_reverse_edges(graph) achieves similar result
# sample_g=dgl.add_reverse_edges(sample_g)

# print graph properties
print("Node IDs are: \n{}\n".format(sample_g.nodes()))
print("Source & destination nodes of every edge are: \n{}\n".format(sample_g.edges()))
print("Edge IDs for directed graph are: \n{}\n".format(sample_g.edge_ids(sample_u, sample_v)))

# print all edges
print("Edge IDs for bidirectional/undirected graph are: \n{}".format(sample_g.edge_ids(sample_u+sample_v, sample_v+sample_u)))

In [ ]:
# DO NOT CHANGE THIS CELL
# find all edges connected to node 1
node_id=1
print("Source node(s) {0[0]} are connected to destination node(s) {0[1]}. ".format(sample_g.in_edges(node_id)))

In [ ]:
# DO NOT CHANGE THIS CELL
# get node degrees
print("Node degrees for all nodes: \n{}".format(sample_g.in_degrees()))

In [ ]:
# DO NOT CHANGE THIS CELL
# visualize the graph
import networkx as nx

# draw plot using networkx
G=dgl.to_networkx(sample_g)
nx.draw_networkx(G)

_注意：此图由 5 个节点和 3 条边组成。节点数量是根据给定边中的最大节点 ID 自动推断出来的。此外，边 ID 将根据边的添加顺序自动编号，对于双向/无向图，边将进行两次编号。_ 

<a name='s1-e1'></a>
### 练习 #1 - 构建一个简单的图 ###
我们来创建下方的图。

<p><img src='images/sample_graph_2.png' width=240></p>

**说明**：<br>
* 仅修改 `<FIXME>` 以创建示例图。

In [ ]:
# create sources nodes and destination nodes
u=<<<<FIXME>>>>
v=<<<<FIXME>>>>
g=dgl.graph((u, v))

# when making undirected graphs, edges are treated as bidirectional
g=dgl.to_bidirected(g)

# draw plot using networkx
nx.draw_networkx(dgl.to_networkx(g))

单击 ... 即可显示**解决方案**。

<a name='s1-1.2'></a>
### 处理节点特征和边特征 ###
许多图数据都包含节点和边特征。我们可以通过 [`dgl.DGLGraph.ndata`](https://docs.dgl.ai/en/latest/generated/dgl.DGLGraph.ndata.html#dgl.DGLGraph.ndata) 和 [`dgl.DGLGraph.edata`](https://docs.dgl.ai/en/latest/generated/dgl.DGLGraph.edata.html#dgl.DGLGraph.edata) 接口分配和检索节点和边特征，类似于在 [Python 字典](https://docs.python.org/3/tutorial/datastructures.html#dictionaries) 中添加/检索键值对的方式。`ndata` 和 `edata` 还可以用于存储 `labels` 和 `train`/`test masks`等深度学习的其他节点级和边级数据。此外，还有关于整个图的全局属性。

_注意：DGLGraph 仅接受存储为数值[张量](https://pytorch.org/docs/stable/tensors.html)的特征。随着深度学习的广泛发展，我们可以通过多种方式将各类属性编码为数值特征。_ 

In [ ]:
# DO NOT CHANGE THIS CELL
# print ndata
print("Node data: \n{}\n".format(g.ndata))

# print edata
print("Edge data: \n{}".format(g.edata))

下面，我们演示如何将随机值分配为节点特征向量。我们使用 [`dgl.DGLGraph.num_nodes()`](https://docs.dgl.ai/generated/dgl.DGLGraph.num_nodes.html#dgl.DGLGraph.num_nodes) 函数来获取图中的节点数量，然后使用 `dgl.DGLGraph.ndata` 分配一个名为 `feat` 的随机多维节点特征向量。我们也可以使用 [`dgl.DGLGraph.num_edges()`](https://docs.dgl.ai/en/0.8.x/generated/dgl.DGLGraph.num_edges.html?highlight=num%20edges#dgl.DGLGraph.num_edges) 和 `dgl.DGLGraph.edata` 进行类似的分配。

_注意：节点和边特征可以随意命名，就像我们为 Python 字典的密钥任意命名一样。例如，我们可以将节点特征命名为 `f_n`，而非 `feat`。_ 

In [ ]:
# DO NOT CHANGE THIS CELL
# get number of nodes
num_nodes=g.num_nodes()

# assign a 4-dimensional random node feature vector called feat for each node
g.ndata['feat']=torch.randn(num_nodes, 4)

# print node features
print("Node features ({}): \n{}\n".format(g.ndata['feat'].shape, g.ndata))

# assign a 5-dimensional random edge feature vector called f_e for each edge
num_edges=g.num_edges()
g.edata['f_e']=torch.randn(num_edges, 5)
print("Edge features ({}): \n{}".format(g.edata['f_e'].shape, g.edata))

<a name='s1-1.3'></a>
### 图数据的表示 ###
图通常以**邻接矩阵**表示。如果一个图有 `n` 个节点，那么邻接矩阵的维度就是 `n` x `n`。矩阵包含 `取值 0 或 1 的` 向量，以指示源节点和目标节点之间是否存在连接。虽然 [`dgl.DGLGraph.adj()`](https://docs.dgl.ai/en/0.8.x/generated/dgl.DGLGraph.adj.html?highlight=adj) 返回的是[稀疏张量](https://pytorch.org/docs/stable/sparse.html)，但我们可以进一步将其转换为稠密张量，使其在视觉上更加直观。在我们的示例图中，邻接矩阵如下所示：

<p><img src='images/adj_matrix.png' width=240></p>

不过，尽管将图结构可视化为邻接矩阵，可能非常方便且直观，但如果它是稀疏张量（充满零），这通常不是最高效的做法。**邻接表**是我们表示边的另一种方式。它们将节点之间的边连接性描述为一维张量 `(v, u)` 的元组，以表示所有边的目标节点和源节点。

In [ ]:
# DO NOT CHANGE THIS CELL
# print adjacency matrix as a sparse Tensor
print("Adjacency matrix (sparse Tensor): \n{}\n".format(g.adj()))

# print adjacency matrix as a dense Tensor
print("Adjacency matrix (dense Tensor): \n{}".format(g.adj().to_dense()))

In [ ]:
# DO NOT CHANGE THIS CELL
# manually calculate node degrees and use .long() to convert to integers
g.ndata['degree']=g.adj().to_dense().sum(axis=1).long()

# print node degrees
print("Node degrees computed manually: \n{}\n".format(g.ndata['degree']))

# node degrees can also be obtained via DGL API
print("Node degrees via DGL API: \n{}".format(g.in_degrees()))

# features can also be deleted, analogous to Dictionary entries
del g.ndata['degree']

In [ ]:
# DO NOT CHANGE THIS CELL
# print destination and source nodes for all edges as adjacency lists
u=g.edges()[0].tolist()
v=g.edges()[1].tolist()
print("Edges: \n{}".format(list(zip(u, v))))

<a name='s1-2'></a>
## 数据集概述 ##
为确保演示贴合实际，我们将使用来自 [**O**pen **G**raph **B**enchmark](https://ogb.stanford.edu/) 的 [`ogbn-arxiv`](https://ogb.stanford.edu/docs/nodeprop/#ogbn-arxiv) 数据集。`ogbn-arxiv` 数据集是一个有向图，表示计算机科学(CS) arXiv 文献之间的引文网络。每个节点是一篇 arXiv 文献，每个有向边表示一篇文献引用了另一篇文献。每篇文献都有一个 128 维 [word2vec](https://en.wikipedia.org/wiki/Word2vec) 特征向量。这个示例很好地说明了如何使用 GNN 利用图中嵌入的信息进行预测。

每个节点都能归入 arXiv CS 文献的 40 个主题之一，例如 cs.AI、cs.LG 和 cs.OS，具体由文献的作者和 arXiv 审核人手动确定和标记。

虽然某些数据集可能包含多个图，但 `ogbn-arxiv` 数据集只包含一个图，且起始位置为 `0`。

In [ ]:
# DO NOT CHANGE THIS CELL
# import dependencies
from ogb.nodeproppred import DglNodePropPredDataset

# load data
dataset=DglNodePropPredDataset(name='ogbn-arxiv')

# assign graph and labels
g, labels=dataset[0]
print(g)

<a name='s1-2.1'></a>
### 探索性地分析数据 ###

为进一步了解数据集，我们将执行一些基本的探索性数据分析。

In [ ]:
# DO NOT CHANGE THIS CELL
# print node keys
print("Node dict keys: \n{}\n".format(g.ndata.keys()))

# print node feature shape
print("Node feature shape (num_of_nodes x num_of_features): \n{}\n".format(g.ndata['feat'].shape))

# print number of nodes
print("Number of nodes: \n{}\n".format(g.num_nodes()))
      
# print number of edges
print("Number of edges: \n{}".format(g.num_edges()))

In [ ]:
# DO NOT CHANGE THIS CELL
# print labels
print("Labels shape: \n{}\n".format(labels.shape))
print("Label classes: \n{}".format(labels.unique()))

<a name='s1-e2'></a>
### 练习 #2 - 查找连接数最多的节点 ###
我们来查找连接数（即度）最高的节点。

**说明**：<br>
* 仅修改 `<FIXME>` 以查找连接数最多的节点。

In [ ]:
# placeholder for max node id and max connections count
max_node_id=0
max_count=0

# iterate through all nodes
for each_node in g.nodes(): 
    # check if number of connections is larger than current max
    count=len(g.<<<<FIXME>>>>(each_node)[0])
    if count>max_count: 
        
        # set max_count and max_node_id
        max_count=count
        max_node_id=each_node

# print node with most connections
print("Node_{} has the most connections. ".format(max_node_id))

单击 ... 即可显示**解决方案**。

In [ ]:
# DO NOT CHANGE THIS CELL
# Node degrees can also be accessed via `.in_degrees()` or `.out_degrees()`
print("Node degrees via DGL API: \n{}\n".format(g.in_degrees()))

# print node with most connections
print("Node_{} has the most connections. ".format(torch.argmax(g.in_degrees())))

<a name='s1-2.2'></a>
### 数据准备和子图 ###
在使用数据集进行机器学习之前，我们将执行数据拆分以验证模型。原始数据集包含 169,343 个节点和 1,116,243 条边。数据拆分的依据是文献发表日期。通过在发表时间较早的文献上训练模型，我们随后可以使用该模型预测新发表的文献的标签。这可为 arXiv 审核人带来巨大的价值。鉴于数据所包含的历史范围，我们将 2017 年及之前发表的文献作为训练集，2018 年发表的文献作为验证集，2019 年及之后发表的文献作为测试集。OGB 数据加载器可为我们直接、便利地提供`train`（训练集）、`valid`（验证集） 和 `test`（测试集）的筛选掩码。

我们首先使用一个数据子集进行演示。具体而言，由于我们要使用密集邻接矩阵来构建第一个 GNN，我们的内存容量仅够装下节点的子集。DGL 提供了一个便捷的 [`dgl.DGLGraph.subgraph(nodes)`](https://docs.dgl.ai/en/0.2.x/generated/dgl.DGLGraph.subgraph.html) 函数来帮助我们实现这一目标。通过查看新图中的节点特征 `dgl.NID` 或边特征 `dgl.EID`，我们可以获得从子图映射到原图的节点/边。此外，`subgraph` 会将初始特征复制到子图中。在了解 GNN 的工作原理后，我们将能使用稀疏邻接矩阵训练整个数据集。

_注意：尽管 `ogbn-arxiv` 数据集表示有向图，但我们会忽略边方向并将其视为无向。_ 

In [ ]:
# DO NOT CHANGE THIS CELL
# import dependencies
import numpy as np

# assign number of labels
num_classes=6

# get subset of node indices
sub_nodes=np.where(np.isin(labels, range(num_classes)))[0]

# get subset of labels
sub_labels=labels[np.isin(labels, range(num_classes))]

# get subgraph and make bidirectional/undirected
sub_g=g.subgraph(sub_nodes)
sub_g=dgl.to_bidirected(sub_g, copy_ndata=True)
print(sub_g)

# print parent node IDs
parent_nodes=sub_g.ndata[dgl.NID]
print("Parent node IDs: {}".format(parent_nodes))

In [ ]:
# DO NOT CHANGE THIS CELL
# get data split
split_idx=dataset.get_idx_split()

# get train, valid, and test splits
train_idx, valid_idx, test_idx=split_idx["train"], split_idx["valid"], split_idx["test"]

# DO NOT CHANGE THIS CELL
# sample subset for the train, valid, and test set using parent_nodes
train_mask=[True if idx in train_idx else False for idx in parent_nodes]
valid_mask=[True if idx in valid_idx else False for idx in parent_nodes]
test_mask=[True if idx in test_idx else False for idx in parent_nodes]

print("{} nodes for training: \n{} nodes for validation: \n{} nodes for testing. ".format(sum(train_mask), sum(valid_mask), sum(test_mask)))

<a name='s1-3'></a>
## 构建用于节点分类的图神经网络 ##
图数据颇具挑战性，因为标准的深度学习方法主要关注结构化数据，例如固定大小的像素网格（图像）和序列（文本）。图神经网络 (**GNN**) 是指在图上应用深度学习的各种不同方法：
* 充分利用图结构
* 根据图大小及其特征来考量可扩展性和效率
* 提供轻松完成节点级、边级和图级预测任务的方法。

<a name='s1-3.1'></a>
### 消息传递和图卷积 ###
**消息传递**是指图中每个节点将自身信息发送给邻域节点，并接收它们回复的消息，以更新其状态并了解其环境。它使图神经网络能够通过关联邻域节点发送的消息（特征）来更新节点的特征，从而探索图的连接性。消息传递功能是 DGL 库的核心，DGL 中的大多数图计算都有赖于此功能。消息传递可视为两个主要步骤：
1. 第一步包括"消息"阶段，即节点将消息发送或散布到其邻域节点处。执行对象是图中所有的边或者相关的边。
2. 第二步是"化简"阶段，其中消息由接收节点进行**聚合**，并用于更新其特征。执行对象是图中所有的节点或相关的节点。

**图[卷积](https://en.wikipedia.org/wiki/Convolution)**结合了来自邻域的信息，并将更新的节点特征编码为*潜隐表示*。它可基于简单的消息传递机制实现，其中涉及邻域特征的线性组合，且聚合使用的权重仅取决于图结构。

* [嵌入](https://en.wikipedia.org/wiki/Embedding)指的是一个相对低维的空间，我们可以将高维向量转换到其中，使相似的项目彼此靠近。

我们将通过不同的节点分类图神经网络来演示这些基本机制。简而言之，节点分类是指通过评估邻域节点的特征和信息来预测特定节点的标签的任务。

<p><img src='images/gnn_node_classification.png' width=720></p>

<a name='s1-4'></a>
## 使用 PyTorch 构建 GNN ##
[PyTorch](https://pytorch.org/) 是一个基于 Python 编程语言和 [Torch](http://torch.ch/) 库的开源机器学习框架。它旨在用于创建深度神经网络，非常适合深度学习研究。我们首先使用 PyTorch 来定义图神经网络的层，以演示基本机制。稍后我们将在实验中探索构建图神经网络的其他方法。

下面，我们将通过定义一个简单的双层图神经网络，来探索各种方法。第一层将矩阵乘法应用到特征向量、邻接矩阵以及权重矩阵上。我们还会将生成的矩阵传递给深度学习的非线性激活函数。此层将允许我们实施 `sum-pooling` 和 `mean-pooling` 等各种聚合方案。

除了 GNN 模型之外，我们还将定义一个评估函数和一个训练函数，以便进行标准的深度学习模型训练。由于我们训练的是分类模型，因此我们将计算[负对数似然损失](https://pytorch.org/docs/stable/generated/torch.nn.NLLLoss.html) (`nll_loss`) 作为损失，并将准确率（`accuracy`）作为评估指标。

<p><img src='images/aggregation.png' width=960></p>

In [ ]:
# DO NOT CHANGE THIS CELL
# import dependencies
import torch.nn as nn
import torch.nn.functional as F

In [ ]:
# DO NOT CHANGE THIS CELL
# define SimpleGraphNet
class SimpleGraphNet(nn.Module):
    """Simple graph neural network
    
    Parameters
    ----------
    in_feats (int): input feature size
    h_feats (int): hidden feature size
    num_classes (int): number of classes
    """
    def __init__(self, in_feats, h_feats, num_classes):
        # for inheritance we use super() to refer to the base class
        super(SimpleGraphNet, self).__init__()
        
        # two linear layers where each one will have its own weights, W
        # first layer computes the hidden layer
        self.layer1 = nn.Linear(in_feats, h_feats)
        # use num_classes units for the second layer to compute the classification of each node
        self.layer2 = nn.Linear(h_feats, num_classes)

    def forward(self, g, h, adj): 
        """Forward computation
        
        Parameters
        ----------
        g (DGLGraph): the input graph
        h (Tensor): the input node features
        adj (Tensor): the graph adjacency matrix
        """
        # apply first linear layer's transform weights 
        x=self.layer1(h)
        
        # perform matrix multiplication with the adjacency matrix and node features to 
        # aggregate/recombine across neighborhoods
        x=torch.mm(adj, x)
        
        # apply a relu activation function
        x=F.relu(x)
        
        # apply second linear layer's transform weights
        x=self.layer2(x)
        return x

In [ ]:
# DO NOT CHANGE THIS CELL
# import dependencies
import time

# define evaluate
def evaluate(model, g, adj, labels, mask):
    """Model evaluation for particular set

    Parameters
    ----------
    model (nn.Module): the model
    features (Tensor): the feature tensor
    adj (Tensor): the graph adjacency matrix
    labels (Tensor): the ground truth labels
    mask (Tensor): the mask for a specific subset
    """
    # assign features
    features=g.ndata['feat']
    
    # set to evaluation mode
    model.eval()
    
    with torch.no_grad():
        # put features through model to obtain logits 
        logits=model(g, features, adj)
        
        # get logits and labels for particular set
        logits=logits[mask]
        labels=labels[mask]
        
        # get most likely class and count the number of corrects
        _, indices = torch.max(logits, dim=1)
        correct = torch.sum(indices == labels)
        
        # return accuracy
        return correct.item() * 1.0 / len(labels)

In [ ]:
# DO NOT CHANGE THIS CELL
# define train
def train(model, g, adj, labels):
    """Model training

    Parameters
    ----------
    model (nn.Module): the model
    features (Tensor): the feature tensor
    adj (Tensor): the graph adjacency matrix
    labels (Tensor): the ground truth labels
    """
    # assign features
    features=g.ndata['feat']
    
    # use a standard optimization pipeline using the adam optimizer
    optimizer=torch.optim.Adam(model.parameters(), lr=0.02)

    # standard training pipeline with early stopping
    best_acc=0.0
    for epoch in range(200): 
        start=time.time()

        # set to training mode
        model.train()
        
        # forward step
        # calculate logits and loss
        logits=model(g, features, adj)
        # calculate loss using log_softmax and negative log likelihood
        logp=F.log_softmax(logits, 1)
        loss=F.nll_loss(logp[train_mask], labels[train_mask])

        # backward step
        # zero out gradients before accumulating the gradients on backward pass
        optimizer.zero_grad()
        loss.backward()
        
        # apply the optimizer to the gradients
        optimizer.step()
        
        # evaluate on validation and test sets
        val_acc=evaluate(model, g, adj, labels, valid_mask)
        test_acc=evaluate(model, g, adj, labels, test_mask)
        
        # compare validation accuracy with best accuracy at 10 epoch intervals, which will update if exceeded
        if (epoch%10==0) & (val_acc>best_acc):
            best_acc=val_acc
            print("Epoch {:03d} | Loss {:.4f} | Validation Acc {:.4f} | Test Acc {:.4f} | Time(s) {:.4f}".format(
                epoch, loss.item(), val_acc, test_acc, time.time()-start))

<a name='s1-4.1'></a>
### 求和池化 ###
作为切入点，我们将使用 `sum-pooling` 聚合训练简单的图神经网络，以执行节点分类。回想一下，`sum-pooling` 可能会导致与特征扩展相关的问题，并且可能无法给出最佳结果。

<p><img src='images/sum-pooling.PNG' width=240></p>

In [ ]:
# DO NOT CHANGE THIS CELL
# create adjacency matrix
adj=sub_g.adj().to_dense()

# modify the adjacency matrix by adding the identity matrix to ensure nodes consider their own features
adj=adj+torch.eye(sub_g.adj().shape[0])

In [ ]:
# DO NOT CHANGE THIS CELL
# instantiate simple model
model=SimpleGraphNet(sub_g.ndata['feat'].shape[1], 32, num_classes)

# print model architecture
print(model)

# start training
train(model, sub_g, adj, sub_labels)

<a name='s1-4.2'></a>
### 基准 MLP 模型 ###
我们可以使用[单位矩阵](https://en.wikipedia.org/wiki/Identity_matrix)来测试模型，而非邻接矩阵。这相当于创建一个标准的 MLP 分类模型，在不同顶点上共享权重。我们可以将此作为基准，了解图卷积的改进程度。

In [ ]:
# DO NOT CHANGE THIS CELL
# modify the adjacency matrix by adding the identity matrix to ensure nodes consider their own features
adj=torch.eye(sub_g.adj().shape[0])

In [ ]:
# DO NOT CHANGE THIS CELL
# instantiate simple model
model=SimpleGraphNet(sub_g.ndata['feat'].shape[1], 32, num_classes)

# print model architecture
print(model)

# start training
train(model, sub_g, adj, sub_labels)

<a name='s1-e3'></a>
### 练习  #3 - 平均池化 ###
使用 `mean-pooling` 聚合的图神经网络会对向量进行归一化，以防止特征呈爆炸式增长，因为输出特征的规模可能会有所增大。

<p><img src='images/mean-pooling.PNG' width=240></p>

**说明**：<br>
* 仅修改 `<FIXME>` 以计算每个节点的连接数。
* 执行以下单元，使用 `mean-pooling` 聚合训练用于节点分类的 GNN。

In [ ]:
# create adjacency matrix
adj=sub_g.adj().to_dense()

# modify the adjacency matrix by adding the identity matrix to ensure nodes consider their own features
adj=adj+torch.eye(sub_g.adj().shape[0])

# get node degrees
deg=<<<<FIXME>>>>

# divide the adjacency matrix by the degree matrix, which is equivalent to multiplying it with the 
# inverse of the degree matrix. This gives a normalize propagation rule, which should hopefully deal with 
# any exploding signal that we might have
adj=adj/deg

In [ ]:
# DO NOT CHANGE THIS CELL
# instantiate simple model
model=SimpleGraphNet(sub_g.ndata['feat'].shape[1], 32, num_classes)

# print model architecture
print(model)

# start training
train(model, sub_g, adj, sub_labels)

单击 ... 即可显示**解决方案**。

<a name='s1-4.3'></a>
### 图卷积网络 (GCN) ###
正如 Kipf 和 Welling ([arXiv](https://arxiv.org/abs/1609.02907)) 指出的那样，常被引用的节点分类图卷积网络 (GCN) 在更新规则中使用对称归一化的方法。它涉及将可学习的函数两端乘以度数矩阵的平方根的倒数，这相当于除以节点的邻域大小和相邻节点的邻域大小的乘积的平方根。

<p><img src='images/gcn.PNG' width=240></p>

In [ ]:
# DO NOT CHANGE THIS CELL
# create adjacency matrix
adj=sub_g.adj().to_dense()

# modify the adjacency matrix by adding the identity matrix to ensure nodes consider their own features
adj=adj+torch.eye(sub_g.adj().shape[0])

# get node degrees
deg=adj.sum(dim=0)

# normalization computes 1 over the square root of the degree matrix
# multiply that on both sides with the adjacency matrix
norm_deg=torch.diag(1.0/torch.sqrt(deg))
# get the normalized adjacency matrix by multiplying the normalized degree matrix with 
# the product of the adjacency matrix and the normalized degree matrix
norm_adj=torch.mm(norm_deg, torch.matmul(adj, norm_deg))

In [ ]:
# DO NOT CHANGE THIS CELL
# instantiate simple model
model=SimpleGraphNet(sub_g.ndata['feat'].shape[1], 32, num_classes)

# print model architecture
print(model)

# start training
train(model, sub_g, norm_adj, sub_labels)

<a name='s1-5'></a>
## 使用 DGL 内置模组构建 GNN ##
DGL 提供了许多热门 GNN 层的实现。它们都可以用一行代码轻松调用。如需查看受支持的图卷积模组的完整列表，请单击[此处](https://docs.dgl.ai/api/python/nn-pytorch.html)。

<a name='s1-5.1'></a>
### GraphConv ###
接下来，我们将使用 DGL 中的 [GraphConv](https://docs.dgl.ai/en/0.9.x/generated/dgl.nn.pytorch.conv.GraphConv.html) 模组实现一个 3 层 GCN，该 GCN 能像上文一样利用均值归一化。通过堆叠 `N个` GCN 层，特征表示会更新节点信息，至高可达 `N` 跳。N 通常被视为模型调优的超参数。

_注意：对于现在这种简单的方法，我们只需使用 `GraphConv` 中的 `in_feats`、`out_feats` 和 `norm` 参数。此外，所有 3 层都使用相同长度的 `h_feat` 隐藏状态向量。对于 `norm` 参数，我们可以使用 `right` 将聚合信息除以每个节点的入度，这相当对所接收的信息进行平均。或者，如果不应用归一化，我们可以使用 `none`；如果使用对称归一化扩展信息，则使用 `both`（默认）。_ 

In [ ]:
# DO NOT CHANGE THIS CELL
# import dependencies
from dgl.nn import GraphConv

# define GCN model
class BuiltinGCN(nn.Module):
    """Graph convolutional network using DGL supported graph convolution modules
    
    Parameters
    ----------
    in_feats (int): input feature size
    h_feats (int): hidden feature size
    num_classes (int): number of classes
    """
    def __init__(self, in_feat, h_feat, num_classes):
        super(BuiltinGCN, self).__init__()
        self.layer1=GraphConv(in_feat, h_feat, norm='right')
        self.layer2=GraphConv(h_feat, h_feat, norm='right')
        self.layer3=GraphConv(h_feat, num_classes, norm='right')

    def forward(self, g, h):
        """Forward computation
        
        Parameters
        ----------
        g (DGLGraph): the input graph
        features (Tensor): the input node features
        """
        h=self.layer1(g, h)
        h=F.relu(h)
        h=self.layer2(g, h)
        h=F.relu(h)
        h=self.layer3(g, h)
        return h

In [ ]:
# DO NOT CHANGE THIS CELL
# import dependencies
import time

# define evaluate
def evaluate(model, g, labels, mask):
    """Model evaluation for particular set

    Parameters
    ----------
    model (nn.Module): the model
    g (DGLGraph): the input graph
    labels (Tensor): the ground truth labels
    mask (Tensor): the mask for a specific subset
    """
    # assign features
    features=g.ndata['feat']
    
    # set to evaluation mode
    model.eval()
    
    with torch.no_grad():
        # put features through model to obtain logits 
        logits=model(g, features)
        
        # get logits and labels for particular set
        logits=logits[mask]
        labels=labels[mask]
        
        # get most likely class and count the number of corrects
        _, indices = torch.max(logits, dim=1)
        correct = torch.sum(indices == labels)
        
        # return accuracy
        return correct.item() * 1.0 / len(labels)

In [ ]:
# DO NOT CHANGE THIS CELL
# define train
def train(model, g, labels):
    """Model training

    Parameters
    ----------
    model (nn.Module): the model
    features (Tensor): the feature tensor
    labels (Tensor): the ground truth labels
    """
    # assign features
    features=g.ndata['feat']
    
    # use a standard optimization pipeline using the adam optimizer
    optimizer=torch.optim.Adam(model.parameters(), lr=0.02)

    # standard training pipeline with early stopping
    best_acc=0.0
    for epoch in range(200): 
        start=time.time()

        # set to training mode
        model.train()
        
        # forward step
        # calculate logits and loss
        logits=model(g, features)
        # calculate loss using log_softmax and negative log likelihood
        logp=F.log_softmax(logits, 1)
        loss=F.nll_loss(logp[train_mask], labels[train_mask])

        # backward step
        # zero out gradients before accumulating the gradients on backward pass
        optimizer.zero_grad()
        loss.backward()
        
        # apply the optimizer to the gradients
        optimizer.step()
        
        # evaluate on validation and test sets
        val_acc=evaluate(model, g, labels, valid_mask)
        test_acc=evaluate(model, g, labels, test_mask)
        
        # compare validation accuracy with best accuracy at 10 epoch intervals, which will update if exceeded
        if (epoch%10==0) & (val_acc>best_acc):
            best_acc=val_acc
            print("Epoch {:03d} | Loss {:.4f} | Validation Acc {:.4f} | Test Acc {:.4f} | Time(s) {:.4f}".format(
                epoch, loss.item(), val_acc, test_acc, time.time()-start))

In [ ]:
# DO NOT CHANGE THIS CELL
# instantiate GNN model using built-in GraphConv layers
model=BuiltinGCN(sub_g.ndata['feat'].shape[1], 32, len(labels.unique()))

# add self-loop to ensure nodes consider their own features
sub_g=dgl.add_self_loop(sub_g)

# print model architecture
print(model)

# start training
train(model, sub_g, sub_labels)

**非常棒！**您已顺利完成培训。

<a href="https://www.nvidia.com/dli"> <img src="images/DLI_Header.png" alt="Header" style="width: 400px;"/> </a>